In [8]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import requests
import io
import time

In [9]:
def fetch_and_load_dataset():
    """
    Tải và xử lý dữ liệu mạng các trang Facebook về thực phẩm
    Returns:
        DataFrame: Chứa các cạnh của đồ thị
    """
    print("Đang tải dữ liệu mạng Facebook Food Pages...")

    # URL của dataset
    dataset_link = "https://nrvis.com/download/data/soc/fb-pages-food.zip"

    try:
        # Tải dữ liệu
        response = requests.get(dataset_link)
        if response.status_code == 200:
            # Đọc file zip
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))

            # Đọc dữ liệu từ file trong zip
            edges_data = pd.read_csv(zip_file.open('fb-pages-food.edges'),
                                     sep=' ',
                                     names=['start_node', 'end_node'])
            return edges_data
        else:
            raise Exception("Không thể tải dữ liệu")
    except Exception as error:
        print(f"Lỗi khi tải dữ liệu: {error}")
        return None

In [10]:
def evaluate_network(graph):
    """
    Phân tích đặc trưng của mạng Facebook Pages
    Args:
        graph: Đồ thị NetworkX
    Returns:
        dict: Các số đo của mạng
    """
    analysis_results = {}

    # 1. Thông tin cơ bản
    print("\nĐang tính toán thông tin cơ bản...")
    analysis_results['total_nodes'] = graph.number_of_nodes()
    analysis_results['total_edges'] = graph.number_of_edges()
    analysis_results['graph_density'] = nx.density(graph)

    # 2. Degree Analysis
    print("Đang tính toán độ liên kết (degree metrics)...")
    start_timer = time.time()
    node_degrees = dict(graph.degree())
    analysis_results['mean_degree'] = np.mean(list(node_degrees.values()))
    analysis_results['max_degree'] = max(node_degrees.values())
    analysis_results['all_degrees'] = node_degrees
    print(f"Thời gian hoàn tất: {time.time() - start_timer:.2f}s")

    # 3. Degree Centrality
    print("Đang tính toán Degree Centrality...")
    start_timer = time.time()
    degree_centrality = nx.degree_centrality(graph)
    analysis_results['degree_centrality'] = {
        'values': degree_centrality,
        'max_value': max(degree_centrality.values()),
        'mean_value': sum(degree_centrality.values()) / len(degree_centrality),
        'top_nodes': sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian hoàn tất: {time.time() - start_timer:.2f}s")

    # 4. Betweenness Centrality
    print("Đang tính toán Betweenness Centrality...")
    start_timer = time.time()
    betweenness_centrality = nx.betweenness_centrality(graph)
    analysis_results['betweenness_centrality'] = {
        'values': betweenness_centrality,
        'max_value': max(betweenness_centrality.values()),
        'mean_value': sum(betweenness_centrality.values()) / len(betweenness_centrality),
        'top_nodes': sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian hoàn tất: {time.time() - start_timer:.2f}s")

    # 5. Closeness Centrality
    print("Đang tính toán Closeness Centrality...")
    start_timer = time.time()
    closeness_centrality = nx.closeness_centrality(graph)
    analysis_results['closeness_centrality'] = {
        'values': closeness_centrality,
        'max_value': max(closeness_centrality.values()),
        'mean_value': sum(closeness_centrality.values()) / len(closeness_centrality),
        'top_nodes': sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian hoàn tất: {time.time() - start_timer:.2f}s")

    # 6. PageRank
    print("Đang tính toán PageRank...")
    start_timer = time.time()
    page_rank = nx.pagerank(graph, alpha=0.85)
    analysis_results['pagerank'] = {
        'values': page_rank,
        'max_value': max(page_rank.values()),
        'mean_value': sum(page_rank.values()) / len(page_rank),
        'top_nodes': sorted(page_rank.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian hoàn tất: {time.time() - start_timer:.2f}s")

    return analysis_results

In [11]:
def summarize_results(analysis_results):
    """
    In kết quả phân tích đồ thị
    """
    print("\nTỔNG QUAN PHÂN TÍCH MẠNG FACEBOOK FOOD PAGES")
    print("=" * 50)

    print("\n1. THÔNG TIN CƠ BẢN")
    print("-" * 30)
    print(f"Tổng số nút: {analysis_results['total_nodes']:,}")
    print(f"Tổng số cạnh: {analysis_results['total_edges']:,}")
    print(f"Mật độ đồ thị: {analysis_results['graph_density']:.6f}")
    print(f"Liên kết trung bình mỗi nút: {analysis_results['mean_degree']:.2f}")
    print(f"Liên kết nhiều nhất: {analysis_results['max_degree']}")

    print("\n2. PHÂN TÍCH CHỈ SỐ CENTRALITY")
    centrality_metrics = {
        'Degree Centrality': 'degree_centrality',
        'Betweenness Centrality': 'betweenness_centrality',
        'Closeness Centrality': 'closeness_centrality',
        'PageRank': 'pagerank'
    }

    for metric_name, key in centrality_metrics.items():
        print(f"\n{metric_name}:")
        print(f"- Giá trị trung bình: {analysis_results[key]['mean_value']:.6f}")
        print(f"- Giá trị lớn nhất: {analysis_results[key]['max_value']:.6f}")
        print("\nTop 5 nút có giá trị cao nhất:")
        for node, score in analysis_results[key]['top_nodes'][:5]:
            print(f"Nút {node}: {score:.6f}")

In [12]:
# Tải và xử lý dữ liệu
edge_dataframe = fetch_and_load_dataset()

# Tạo đồ thị có hướng từ dữ liệu
print("Đang xây dựng đồ thị...")
directed_graph = nx.from_pandas_edgelist(edge_dataframe, 'start_node', 'end_node', create_using=nx.DiGraph())

Đang tải dữ liệu mạng Facebook Food Pages...
Đang xây dựng đồ thị...


In [13]:
# Phân tích mạng
analysis_metrics = evaluate_network(directed_graph)


Đang tính toán thông tin cơ bản...
Đang tính toán độ liên kết (degree metrics)...
Thời gian hoàn tất: 0.00s
Đang tính toán Degree Centrality...
Thời gian hoàn tất: 0.00s
Đang tính toán Betweenness Centrality...
Thời gian hoàn tất: 19.18s
Đang tính toán Closeness Centrality...
Thời gian hoàn tất: 0.16s
Đang tính toán PageRank...
Thời gian hoàn tất: 0.01s


In [14]:
# Xuất kết quả
summarize_results(analysis_metrics)


TỔNG QUAN PHÂN TÍCH MẠNG FACEBOOK FOOD PAGES

1. THÔNG TIN CƠ BẢN
------------------------------
Tổng số nút: 4,204
Tổng số cạnh: 2,102
Mật độ đồ thị: 0.000119
Liên kết trung bình mỗi nút: 1.00
Liên kết nhiều nhất: 1

2. PHÂN TÍCH CHỈ SỐ CENTRALITY

Degree Centrality:
- Giá trị trung bình: 0.000238
- Giá trị lớn nhất: 0.000238

Top 5 nút có giá trị cao nhất:
Nút 0,276: 0.000238
Nút nan: 0.000238
Nút 0,58: 0.000238
Nút nan: 0.000238
Nút 0,132: 0.000238

Betweenness Centrality:
- Giá trị trung bình: 0.000000
- Giá trị lớn nhất: 0.000000

Top 5 nút có giá trị cao nhất:
Nút 0,276: 0.000000
Nút nan: 0.000000
Nút 0,58: 0.000000
Nút nan: 0.000000
Nút 0,132: 0.000000

Closeness Centrality:
- Giá trị trung bình: 0.000119
- Giá trị lớn nhất: 0.000238

Top 5 nút có giá trị cao nhất:
Nút nan: 0.000238
Nút nan: 0.000238
Nút nan: 0.000238
Nút nan: 0.000238
Nút nan: 0.000238

PageRank:
- Giá trị trung bình: 0.000238
- Giá trị lớn nhất: 0.000309

Top 5 nút có giá trị cao nhất:
Nút nan: 0.000309
Nút n